In [2]:
import spacy
import re
import json
import coreferee
from spacy.matcher import DependencyMatcher
from spacy.matcher import Matcher
nlp = spacy.load("fr_core_news_lg")
nlp.add_pipe('coreferee') #pour la co-reference
matcher = DependencyMatcher(nlp.vocab)
matcher_t = Matcher(nlp.vocab)

#24.8s

C:\Users\emeld\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'fr_core_news_lg' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
def groupe_nominal (mot, position, doc):

    """
    @input : 
        le mot analysé
        sa position dans la définition
        la définition sous forme de doc

    @output : 
        le groupe nominal dans lequel se situe le mot analysé
        -> c'est-à-dire une liste composé du mot donné en entrée et des adjectifs, noms et compléments qui en dépendent
        la position du dernier mot ajouté à cette liste 

    @informations supplémentaires : 
    les patterns sont dans la fonction car il y a une variable
    durée : 0.1s
    """
    test_pour_charge_fixe = [
        {"RIGHT_ID": "anchor_founded","RIGHT_ATTRS": {"ORTH": mot}},
        {"LEFT_ID": "anchor_founded", "REL_OP": "<","RIGHT_ID": "gn","RIGHT_ATTRS": {"POS": {"IN": ["ADJ", "CC", "X","NOUN"]},"DEP": {"IN": ["amod", "nmod", "conj"]}}}
    ]

    pattern_groupe_nominaux_1 = [
        {"RIGHT_ID": "anchor_founded", "RIGHT_ATTRS": {"ORTH": mot}},
        {"LEFT_ID": "anchor_founded", "REL_OP": ">","RIGHT_ID": "gn","RIGHT_ATTRS": {"POS": {"IN": ["ADJ", "CC", "X","NOUN"]},"DEP": {"IN": ["amod", "nmod", "conj"]}}}
    ]

    pattern_groupe_nominaux_2 = [
        { "RIGHT_ID": "anchor_founded", "RIGHT_ATTRS": {"ORTH": mot}},
        {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "gn","RIGHT_ATTRS": {"POS": {"IN": ["ADJ", "CC", "X","NOUN"]},"DEP": {"IN": ["amod", "nmod", "conj"]}}},
        {"LEFT_ID": "gn","REL_OP": ">","RIGHT_ID": "gn2","RIGHT_ATTRS": {"POS": {"IN": ["ADJ", "CC", "X","NOUN"]},"DEP": {"IN": ["amod", "nmod", "conj"]}}}
    ]

    pattern_groupe_nominaux_verbe = [
        {"RIGHT_ID": "anchor_founded","RIGHT_ATTRS": {"ORTH": mot, "POS":"VERB"}},
        {"LEFT_ID": "anchor_founded","REL_OP": ">", "RIGHT_ID": "gn2","RIGHT_ATTRS": {"DEP": {"IN": ["nmod", "conj", "amod"]}}},
        { "LEFT_ID": "gn2","REL_OP": ">","RIGHT_ID": "gn3","RIGHT_ATTRS": {"DEP": {"IN": ["obj"]}}},
        {"LEFT_ID": "gn3","REL_OP": ">","RIGHT_ID": "gn4","RIGHT_ATTRS":{"DEP": {"IN": ["amod"]}}}
    ]

    #pour avoir le mot principal sous forme de lemme
    mot_ss_forme_doc = nlp(mot)
    for token in mot_ss_forme_doc : 
        el = [token.lemma_]
    
    position_final = position
    matcher = DependencyMatcher(nlp.vocab)

    matcher.add("gn1", [pattern_groupe_nominaux_1])
    matcher.add("gn1fixe", [test_pour_charge_fixe])

    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        if (doc[token_ids[1]].lemma_) not in el :
            if position == token_ids[0]:
                el.append(doc[token_ids[1]].lemma_)
                position_final = token_ids[1]
                if position > position_final : 
                    position_final = position
                

    matcher.remove("gn1")
    matcher.remove("gn1fixe")
    matcher.add("gn2", [pattern_groupe_nominaux_2])
    matches = matcher(doc)
    if len(matches)!=0 : 
        match_id, token_ids = matches[0]
        if position == token_ids[0]:
            
            if (doc[token_ids[1]].lemma_) not in el :
                el.append(doc[token_ids[1]].lemma_)
                position_final = token_ids[1]
                if position > position_final : 
                    position_final = position

            if (doc[token_ids[2]].lemma_) not in el : 
                el.append(doc[token_ids[2]].lemma_)
                if token_ids[2]> position_final : 
                    position_final = token_ids[2]

    matcher.remove("gn2")
    matcher.add("gn4", [pattern_groupe_nominaux_verbe])
    matches = matcher(doc)
    if len(matches)!=0 : #peut être tester ici 
        match_id, token_ids = matches[0]
        if position == token_ids[0]:

            if doc[token_ids[1]].lemma_ not in el:
                el.append(doc[token_ids[1]].lemma_)
                position_final = token_ids[1]
                if position > position_final : 
                    position_final = position

            if doc[token_ids[2]].lemma_ not in el :
                el.append(doc[token_ids[2]].lemma_)
                if token_ids[2]> position_final : 
                    position_final = token_ids[2]

            if doc[token_ids[3]].lemma_ not in el : 
                el.append(doc[token_ids[3]].lemma_)
                if token_ids[3]> position_final : 
                    position_final = token_ids[3]
    return (el,position_final)

#Exemple

doc = nlp("Un stock est un ensemble de produits physiques (marchandises, matières premières ou produits finis) possédés par une entreprise, en attendant d'être utilisés dans le processus de production ou d'être vendus sur un marché.")
mot = "ensemble"
position = 4
groupe_nominal (mot, position, doc)
#(['ensemble', 'produit', 'physique'], 7)


(['ensemble', 'produit', 'physique'], 7)

In [4]:
"""
@input: la position du mot qui a potentiellement un co-référent 
        la définition

@output:si le mot a bien un co-référent, le co-référent 
        sinon, False

Note : ne fonctionne pas pour les "qui"

"""
def coreference (texte,position):
    doc = nlp(texte)
    f = doc._.coref_chains
    #exemple : [0: [1], [6], [24], [32], 1: [11], [21]]
    #tout ce qui est dans 0 a le même référent
    result = []
    for i in range (len(f)):
        for j in range (len(f[i])):
            for k in range (len(f[i][j])):
                x = f[i][j][k]
                if position == x : 
                    if (doc._.coref_chains.resolve(doc[x])) is not None : 
                        result.append(doc._.coref_chains.resolve(doc[x]))

    if len(result)>0:
        return(True,result[0][0])
    else :
        return (False,"")
            
text = "Le cycle d'investissement de l'entreprise inclut toutes les acquisitions d'immobilisations (machines, brevets, etc.). Elles sont destinées à rester dans l'entreprise de façon durable."
coreference(text,21)
#doc[21] = Elles
#(True, acquisitions)

(True, acquisitions)

In [5]:
def qui(doc, position):
    """
    @input: la position du "qui"
            la définition sous forme de doc

    @output: le nom qui précède le "qui"
             la position de ce nom

    @durée : 1.4s
    """
    if len(doc) == 0:
        raise ValueError("Il n'y a pas de texte")

    pos_mot = None
    mot = None

    for sent in doc.sents:
        for token in sent:
            if token.i < position and token.pos_ == "NOUN":
                mot = token.text
                pos_mot = token.i

    if pos_mot is not None:
        return (mot, pos_mot)
    else:
        return ("qui", position)


position = 9
text = "Le cycle de financement regroupe toutes les opérations financières qui concernent l'activité d'une entreprise. On peut le considérer comme la contrepartie du cycle d'investissement et d'exploitation, car il sert à trouver des capitaux pour les financer."
#('opérations', 7)
position = 5
text = "Un stock est un agrégat qui intègre à la période présente des éléments qui étaient déjà présents dans l’agrégat de la période précédente . La notion de stock s’oppose ainsi à celle de flux, qui est un agrégat qui se renouvelle entièrement de période en période . Donc un flux est remis à 0 au début de chaque période, alors qu’un stock a un « effet mémoire » . De manière imagée, on peut dire que le stock est un réservoir d’eau, tandis que le flux correspond à la quantité d’eau ajoutée (flux entrant) ou enlevée (flux sortant) à chaque période . Le stock est donc une accumulation de flux entrants, moins une accumulation de flux sortants"
#('agrégat', 4)
doc = nlp(text)
qui(doc, position)



('agrégat', 4)

In [6]:
def remove_token(doc,emplacement):
    """
        @input: la definition sous forme de doc 
                l'emplacement du mot que l'on veut retirer

        @output:le doc sans le mot que l'on a voulu retirer, toujours sous forme de doc

        """
    tokens = list(doc)
    tokens.pop(emplacement)
    doc_without_token = spacy.tokens.Doc(nlp.vocab, words=[token.text for token in tokens])
    doc_without_token = nlp(doc_without_token.text)
    return(doc_without_token)


In [7]:
def occurrence(mot,doc,token_ids) : 
    """
    #fonction pour retrouver l'emplacement d'un mot dans le doc 
    #fonction implémentée pour retrouver le groupe nominal entier lorsqu'il y a un pp-adj

    @input: la definition sous forme de doc 
            le mot dont on veut retrouver la position
            le token_ids qui corresponds à l'emplacement qu'on connait déjà du mot 

    @output:l'emplacement du mot recherché 


    """
    if type(token_ids)!= int : 
        raise ValueError("Pas d'emplacement donné : /mot/",mot,"/doc/",doc)
    occ =  0
    res = "check"
    for token in doc: 
        #pour savoir le nombre de fois que le mot apparait dans le texte
        if mot == token.text :
            occ += 1
    #si + qu'une fois : 
    if occ > 1 : 
        #pour quand pp_adj avec un mot qui apparait deux fois dans le texte : 
        #par exemple : une charge fixe est une charge
        for token in doc :
            #soit on retrouve le bon emplacement
            if token.text == mot: 
                if (token.i != token_ids):
                    if token.i == token_ids+1:
                        res=(token.i)
                    elif token.i == token_ids-1:
                        res=(token.i)
            else :
                #soit on retourne directement l'emplacement reçu
                reso =(token_ids)
                
        if res == "check" : 
            return (reso)
        else : 
            return (res)
    if occ == 1 :
        for token in doc : 
            if token.text == mot :
                return(token.i)


doc = nlp("Le cycle de financement regroupe les opérations financières qui concernent l'activité d'une entreprise. On peut le considérer comme la contrepartie du cycle d'investissement et d'exploitation, car il sert à trouver des capitaux pour les financer.")
mot = "opérations"
token_ids = 5
occurrence(mot,doc,token_ids)
#6

mot = "cycle" 
texte = "Le cycle de financement regroupe toutes les opérations financières qui concernent l'activité d'une entreprise. On peut le considérer comme la contrepartie du cycle d'investissement et d'exploitation, car il sert à trouver des capitaux pour les financer."
doc = nlp(texte)
token_ids = 2
occurrence(mot,doc,token_ids)
#1

	

1

In [8]:
                
def mots_non_voulus (liste,mot = "relatives"): 
    """
    @input: une liste de mots
            un mot que l'on veut retirer
                qui est "relatives" s'il n'est pas précisé

    @output:la liste donnée en entrée sans le mot que l'on voulait retirer, s'il était bien dans la liste 
            est retournée s'il reste encore au moins un élément dans la liste
            n'est pas retournée -> on retourne False s'il n'y a plus d'élément dans la liste

    """
    for i in liste : 
        if i == mot:
            liste.remove(mot)
    if len(liste)>0:
        return(liste)
    else : 
        return False


In [10]:
"""
@input : 
definition : la définition sous forme de texte 
matcher : le matcher avec tous les patterns choisis ajoutés dans la fonction principale
doc : la définition sous forme de doc 
    /!\ : sans le pp_adj s'il y en 
numéro_sujet : le numéro de l'emplacement du sujet dans le pattern 
numéro_objet : le numéro de l'emplacement de l'objet dans le pattern 
numéro_verbe : le numéro de l'emplacement du verbe dans le pattern 
docu : la définition sous forme de doc 
    /!\ : avec le pp_adj s'il y en
    s'il n'y en a pas, ou s'il n'est juste pas donné en argument , l'argument prit en considération est le doc

@ouput :
un tuple contenant : 
    l'emplacement du verbe dans la phrase 
    puis une liste composée de : 
        eq_el1 : liste du.es sujet.s accompagnés de leur adjectifs / le groupe nominal composant le sujet 
        eq_el2 : liste du.es objet.s accompagnés de leur adjectifs / le groupe nominal composant l'objet 
        position_sujet : celui du premier sujet retrouvé
        position_objet : celui du premier objet retrouvé
"""
def relation(definition,matcher, doc, numéro_sujet, numéro_objet,numéro_verbe,docu=None):
    if docu == None :
        docu = doc
    
    resultat_final = []
    sujet = ""
    objet = ""
    matches = matcher(doc) #on utilise ici le matcher qu'on a importé de la fonction principale 
    if len(matches)!=0 : 
        
        match_id, token_ids = matches[0]
        sujet = (doc[token_ids[numéro_sujet]].text)
        position_sujet = token_ids[numéro_sujet]
        
        if sujet == 'qui' : 
            #modification ici doc -> docu (19)
            sujet = qui(docu,position_sujet)
            position_sujet = sujet[1]
            sujet = sujet[0]


        objet = (doc[token_ids[numéro_objet]].text)
        position_objet = token_ids[numéro_objet]
        if objet == 'qui' : 
            objet = qui(docu,position_objet)
            position_objet = objet[1]
            objet = objet[0]

        #TO = objet sous forme de token
        TO = doc[token_ids[numéro_objet]]
        if TO.text == objet : 
            if TO.pos_ == "PRON":
                objet_co = coreference(definition,position_objet)
                if objet_co[0]== True : 
                    objet= str(objet_co[1])
        #TS = sujet sous forme de token
        TS = doc[token_ids[numéro_sujet]]
        if TS.text == sujet : 
            if TS.pos_ == "PRON":
                sujet_co = coreference(definition,position_sujet)
                if sujet_co[0] == True :
                    sujet= str(sujet_co[1])

        #Pour avoir l'emplacement du sujet et de l'objet s'il y a un pp-adj : 
        position_sujet = occurrence(sujet,docu,position_sujet)
        position_objet = occurrence(objet,docu,position_objet)
        #Pour retrouver les groupes nominaux sujet et objet
        resultat_sujet = groupe_nominal(sujet,position_sujet,docu)
        resultat_objet = groupe_nominal(objet,position_objet,docu)
        
        eq_el1 = resultat_sujet[0]
        eq_el2 = resultat_objet[0]

        #s'assurer qu'il n'y ait pas d'erreurs de type : 
        #"Un stock est un ensemble de produits physiques (marchandises, matières premières ou produits finis) possédés par une entreprise, en attendant d'être utilisés dans le processus de production ou d'être vendus sur un marché."
        #S: '['stock', 'ensemble']' \O: '['ensemble', 'produits', 'physiques']' 
        for i in eq_el1 : 
            if i == eq_el2[0]:
                if i != eq_el1[0]:
                    eq_el1.remove(i)
        
        
        eq_el1 = mots_non_voulus(eq_el1)
        eq_el2 = mots_non_voulus(eq_el2)

        sujet_nlp = nlp(sujet)
        for token in sujet_nlp : 
            sujet = token.lemma_

        objet_nlp = nlp(objet)
        for token in objet_nlp : 
            objet = token.lemma_

        sujet = [sujet]
        objet = [objet]

        resultat_final.append([eq_el1,eq_el2,position_sujet,position_objet,sujet,objet])
        if (eq_el1 == False) or (eq_el2 == False):
                resultat_final.remove([eq_el1,eq_el2,position_sujet,position_objet,sujet,objet])
        numéro_verbe = token_ids[numéro_verbe]

    return(numéro_verbe,resultat_final)

In [11]:
"""
première partie de l'analyse du pp_adj
EX : "produits finis" -> finis n'est reconnu que comme verbe ce qui ne permets pas d'arriver aux analyses voulues car ce participe passé est ici traité comme un adjectif
Nous allons donc chercher un nom suivi de termes dont les lemmes sont des participes passés que l'on retrouve dans le corpus comme apparraissant assez souvent comme adjectifs
@input : la définition
@output :   s'il y a bien un participe passé utilisé comme adjectif, sa position
            sinon : False 
"""
#pour les pp utilisés comme adjectif
def pp_adj(definition):
    matcher = DependencyMatcher(nlp.vocab)
    pp_adj_finir = [
        {"RIGHT_ID": "objet","RIGHT_ATTRS": {"POS":{"IN": ["NOUN"]}}},
        #ajout emprunter + fixe + fixer 
        {"LEFT_ID": "objet","REL_OP": ">","RIGHT_ID": "pp_adj","RIGHT_ATTRS": {"DEP": "acl","LEMMA":{"IN": ["finir","ajouter","emprunter","fixe","fixer","donner"]}}
        }
    ]
    pp_adj_fixe = [
        {"RIGHT_ID": "objet","RIGHT_ATTRS": {"POS":{"IN": ["NOUN"]}}},
        #ex : une charge fixe
        {"LEFT_ID": "objet","REL_OP": "<","RIGHT_ID": "pp_adj_fixe","RIGHT_ATTRS": {"LEMMA":{"IN": ["fixe","fixer"]}}}]
    doc = nlp(definition)
    matcher.add("pp_adj", [pp_adj_finir])
    matcher.add("pp_adj_fixe", [pp_adj_fixe])
    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        return (token_ids[1])
    else : 
        return False
"""
seconde partie de l'analyse du pp_adj
@input :la définition sous forme de doc
        le résultat de la fonction précédente
@output :   si le résultat de la fonction précédente n'est pas fausse 
            la définition sous forme de doc, la relation de dépendance et le pos-tag du pp_adj sont modifiés
"""
def pp_adj2 (doc,pp_adj_position):
    if pp_adj_position is not False : 
        token = doc[pp_adj_position]
        token.pos_ = "ADJ"
        token.dep_ = "amod"
        return(doc)
    else : 
        return False


texte = "Une charge fixe est une charge dont le montant ne varie pas avec le niveau d'activité de l'entreprise."
doc = nlp(texte)
position = pp_adj(texte)
assert position==2


In [12]:
"""
Ces deux fonctions sont similaires aux deux précédentes pp_adj
Nous identifions ici les "tout/tous" employés comme déterminants mais reconnus comme adjectif
@input : la définition
@output :   s'il y a bien un participe passé utilisé comme adjectif, sa position
            sinon : False 
"""
def tous(definition):
    matcher = DependencyMatcher(nlp.vocab)
    pattern_tous = [
        {"RIGHT_ID": "objet","RIGHT_ATTRS": {"POS":{"IN": ["NOUN"]}}},
        {"LEFT_ID": "objet","REL_OP": ">","RIGHT_ID": "tous","RIGHT_ATTRS": {"DEP": "amod","LEMMA":{"IN": ["tous","tout"]}}}]
    
    doc = nlp(definition)
    matcher.add("tous", [pattern_tous])
    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        return (token_ids[1])
    else : 
        return False

def tous2 (doc,tous_position):
    if tous_position is not False : 
        token = doc[tous_position]
        token.pos_ = "DET"
        token.dep_ = "det"
        return(doc)
    else : 
        return False
    
texte = "Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente."
doc = nlp(texte)
position = tous(texte)
assert position == 3

tous2(doc,position)
#Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente.

Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente.

In [13]:
"""
pre_traitement de l'analyse
@input : la définition
         la définition sous forme de doc
@output : 
        docu : la définition sous forme de doc avec la modification appliquée aux potentiels pp_adj et tous
        doc : s'il y a un pp_adj, la définition sous forme de doc, sans le pp_adj afin que l'analyse de la phrase soit le plus correct possible
            @note : nous ne retirons pas le tous employé comme déterminant mais reconnu comme adjectif car sa présence ne modifie pas l'analyse globale de la phrase
"""
def pre_traitement (definition,docu): 
    #On regarde s'il y a un tous utilisé comme adjectif
    position_tous = tous(definition)
    if position_tous is not False : 
        docu = tous2(docu,position_tous) 

    #On regarde s'il y a un verbe au participe passé utilisé comme adjectif (pp-adj)
    position = pp_adj(definition)
    if position is not False : 
        doc = remove_token(docu,position) #On le retire si c'est le cas et doc devient la définition sans le pp-adj
        docu = pp_adj2(docu,position) #docu reste la définition avec le pp-adj dont l'étiquette "POS" devient "ADJ" et "DEP" : "amod" 
    else : 
        doc = docu # s'il n'y a pas de pp-adj doc devient le docu

    return(docu,doc)   

texte = "Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente."
texte = "Une charge fixe est une charge dont le montant ne varie pas avec le niveau d'activité de l'entreprise."

texte_doc = nlp(texte)
pre_traitement(texte,texte_doc)
#(Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente.,
#Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente.)

#(Une charge fixe est une charge dont le montant ne varie pas avec le niveau d'activité de l'entreprise.,
#Une charge est une charge dont le montant ne varie pas avec le niveau d' activité de l' entreprise . )

(Une charge fixe est une charge dont le montant ne varie pas avec le niveau d'activité de l'entreprise.,
 Une charge est une charge dont le montant ne varie pas avec le niveau d' activité de l' entreprise . )

In [14]:

from liste_pattern import pattern_equivalence
liste_pattern_equivalence = pattern_equivalence()
def equivalence (definition,liste=liste_pattern_equivalence) : 
    
    """
    @input : une définition
    @output : Si la définition ne contient pas de relation d'équivalence, retourne "La définition ne contient pas de relation d'entité !"
                Si la définition contient une relation d'équivalence,  retourne de la relation d'équivalence jusqu'à la fin du document 
                permet également d'obtenir le sujet ainsi que l'objet de la relation d'équivalence, 
                sujet = self.eq_el1
                objet = self.eq_el2
    """
    #pattern général
    pattern_equivalence = liste[0]

    #pattern équivalence pour les formes passives du type : 
    # S est constitué de O 
    pattern_equivalence_passif = liste[1]

    #pattern équivalence pour les formes passives du type : 
    # O est appelé S 
    #obl:mod pour sont regroupés dans 
    #xcomp et obj pour sont appelés 
    #,"regrouper"
    pattern_equivalence_passif_2 = liste[2]

    #pattern équivalence pour les formes avec un ADP du type : 
    # S correspond à O 
    pattern_equivalence_ADP = liste[3]

    #pattern équivalence pour les formes avec un ADP et de l'obj partie et qui correspondent donc aux formes : 
    # O font partie de S
    pattern_equivalence_ADP_partie = liste[4]

    #pattern similaire au précédent mais se différence car l'objet et le verbe faire sont séparés par un sous partie de la phrase : 
    # O [V ...]font partie de S
    pattern_equivalence_ADP_partie_2 = liste[5]

    #pattern équivalence pour les formes avec un acl pour le sujet du type : 
    # S englobe O 
    pattern_equivalence_acl = liste[6]

    #pattern similaire au précédent mais se différence car l'objet et le verbe  sont séparés par un sous partie de la phrase : 
    # O [V ...]constituent S
    pattern_equivalence_acl_2 = liste[7]
    

    #pattern pour objet = VERB
    pattern_equivalence_verbe = liste[8]

    #pattern équivalence pour les formes du type : 
    # S représente [,complément,] O 
    #ex :"Un résultat comptable représente, pour une période donnée, le bénéfice ou la perte réalisés par une entité, obtenu en additionnant l’ensemble des produits, auxquels on retranche l’ensemble des charges de l’exercice comptable. Mesure du bénéfice ou de la perte."
    pattern_equivalence_particulier = liste[9]

    #
    equivalence_regrouper_et_appeler = liste[10]

    #pattern équivalence pour le verbe être
    #seul pattern qui n'a pas le verbe comme ancre
    pattern_equivalence_etre = liste[11]
    pattern_equivalence_peutetre = liste[12]
    pattern_equivalence_peutetrecompose= liste[13]

    #On ajoute les matchers qui ont la position de leur sujet de leur objet au même endroit
    matcher = DependencyMatcher(nlp.vocab)
    matcher.add("equivalence", [pattern_equivalence])
    matcher.add("equivalence_al", [pattern_equivalence_acl])
    matcher.add("equivalence_passif", [pattern_equivalence_passif])
    matcher.add("equivalence_adp", [pattern_equivalence_ADP])
    matcher.add("equivalence_adp_partie", [pattern_equivalence_ADP_partie])
    matcher.add("equivalence_verbe", [pattern_equivalence_verbe])
    

    #On retire les tous,toutes,Toutes et Tous de notre définition afin de pouvoir trouver plus facilement les adjectifs qui dépendent des mots-clefs
    #definition =re.sub(r"tous|toutes|Toutes|Tous","", definition)
    #On retire les doubles espaces
    #definition = definition.replace('  ',' ')
    docu = nlp(definition)
    resultat = []
    preT = pre_traitement(definition,docu)
    docu = preT[0]
    doc = preT[1]
    docc = doc

    for token in doc: 
        res =  relation(definition,matcher, doc, 1, 2,0,docu)
        resu = res[1]
        emplacement_verbe = res[0]
        if len(resu)>0 : 
            resultat.append(resu[0])
            doc = doc[emplacement_verbe+1:] #Pour continuer à chercher les autres relations d'équivalence qui se situent après le verbe d'équivalence identifié
        else:
            matcher = DependencyMatcher(nlp.vocab)
            matcher.add("equivalence_adp_partie_2", [pattern_equivalence_ADP_partie_2])
            matcher.add("equivalence_acl_2", [pattern_equivalence_acl_2])
            res =  relation(definition,matcher, doc, 3, 0,2,docu)
            resu = res[1]
            
            emplacement_verbe = res[0]
            if len(resu)>0 : 
                resultat.append(resu[0])
                doc = doc[emplacement_verbe+1:] #Pour continuer à chercher les autres relations d'équivalence qui se situent après le verbe d'équivalence identifié
            else:
                matcher = DependencyMatcher(nlp.vocab)
                matcher.add("equivalence_particulier", [pattern_equivalence_particulier])
                matcher.add("equivalence_peutetre", [pattern_equivalence_peutetre])
                matcher.add("equivalence_peutetrecompose", [pattern_equivalence_peutetrecompose])
                res =  relation(definition,matcher, doc, 1, 3,0,docu)
                resu = res[1]
                emplacement_verbe = res[0]

                if len(resu)>0 : 
                    resultat.append(resu[0])
                    doc = doc[emplacement_verbe+1:]
                else : 
                    matcher = DependencyMatcher(nlp.vocab)
                    #exemple : "Un stock de produits finis est une réserve de biens fabriqués par l'entreprise et prêts à être vendus."
                    
                    matcher.add("equivalence_etre", [pattern_equivalence_etre])
                    res =  relation(definition,matcher, doc, 1,0,2,docu)
                    resu = res[1]
                    emplacement_verbe = res[0]

                    if len(resu)>0 : 
                        resultat.append(resu[0])
                        doc = doc[emplacement_verbe+1:]

                    else :
                        matcher = DependencyMatcher(nlp.vocab)
                        #exemple : "Afin de mener à bien ses activités économiques, commerciales et productives, une entreprise doit effectuer un certain nombre d'opérations qui sont regroupées dans ce que l'on appelle le cycle d'exploitation, qui inclut l'achat de marchandises, leur stockage et leur vente."
                        matcher.add("equivalence_regrouper_et_appeler", [equivalence_regrouper_et_appeler])
                        res =  relation(definition,matcher, doc, 1,4,0,docu)
                        resu = res[1]
                        emplacement_verbe = res[0]

                        if len(resu)>0 : 
                            resultat.append(resu[0])
                            doc = doc[emplacement_verbe+1:]

                        else :
                            matcher = DependencyMatcher(nlp.vocab)
                            #exemple : "Afin de mener à bien ses activités économiques, commerciales et productives, une entreprise doit effectuer un certain nombre d'opérations qui sont regroupées dans ce que l'on appelle le cycle d'exploitation, qui inclut l'achat de marchandises, leur stockage et leur vente."
                            matcher.add("equivalence_passif_2", [pattern_equivalence_passif_2])
                            res =  relation(definition,matcher, doc, 1,2,0,docu)
                            resu = res[1]
                            emplacement_verbe = res[0]

                            if len(resu)>0 : 
                                resultat.append(resu[0])
                                doc = doc[emplacement_verbe+1:]

                            else :
                        
                                #pour lorsqu'il y a plusieurs relations d'équivalence, après ET avant la relation détectée en premier lieu
                                #exemple : 
                                #text = "Le bilan comptable est un document qui synthétise à un moment le patrimoine d’une entreprise. Ce patrimoine comporte un actif (ses avoirs : terrains, machines, stocks...) et un passif (ses ressources : capital, réserves, crédits.). Le bilan est donc une « photographie » du patrimoine de l’entreprise à un moment donné de son histoire."
                                #[[['patrimoine'], ['actif', 'passif'], 18, 21], [['bilan'], ['photographie', 'patrimoine', 'entreprise'], 29, 34], [['bilan', 'comptable'], ['document'], 1, 5]]
                                
                                if len(resultat)>0 : 
                                    j=(resultat[0][3])
                                    if j != None :       
                                        if(emplacement_verbe<j):
                                            if doc.text != docc[:j-1].text:
                                                text = (docc[:j-1]).text
                                                doc = nlp(text)
                                                f = equivalence(text)
                                                if len(f)>0:
                                                    resultat.append(f[0])
                                                    break
    for a in resultat:
        for b in a : 
            if b is not None :
                if type(b) != int : 
                    for c in b :
                        if (c == 'objectif') or (c == 'contrepartie'): 
                            resultat.remove(a)
                        

    return(resultat)

text = "Le cycle de financement englobe toutes les transactions financières qui concernent l'activité d'une entreprise et permettent de trouver des capitaux pour la financer. C'est la contrepartie du cycle d'investissement et d'exploitation."
#[[['cycle', 'financement'], ['transaction', 'financier', 'activité', 'entreprise'], 1, 7]]
text = "Le cycle de financement est l'ensemble des opérations financières qui concernent l'activité d'une entreprise et qui visent à trouver des capitaux pour la financer. Il est le pendant du cycle d'investissement et d'exploitation."
#[[['ensemble', 'opération', 'financier'], ['activité', 'entreprise'], 6, 13], [['cycle', 'financement'], ['ensemble', 'opération', 'financier'], 1, 6]]
text = "Les capitaux propres se composent du capital social, des réserves, des bénéfices non distribués, etc."
#[]
text = "Un BSPCE, ou bon de souscription de parts de créateur d'entreprise, est un titre émis par une société à l'attention de ses dirigeants ou de son personnel, leur permettant d'obtenir des parts du capital de l'entreprise. Les BSPCE fonctionnent de manière similaire aux bons de souscription d'actions, bénéficiant d'un régime fiscal avantageux sous certaines conditions, et sont réservés aux entreprises innovantes ayant été créées depuis moins de 10 ans."
text = "Règles et principes regroupés dans des document représantifs d’une structure pour des parties extérieures et aux yeux de la loi."
#[[['BSPCE', 'bon', 'souscription'], ['titre'], 1, 16]]
text = "des principes regroupés dans le document" #vs des principes qui sont regroupés où là c'est correctement retrouvé

text = "Le stock correspond aux différents flux monétaire entrant et sortant de l'entreprise"
#[[['stock'], ['flux', 'différent'], 1, 5, 'stock', 'flux']]
f=equivalence(text)

print(f[0])
#[['stock'], ['flux', 'différent'], 1, 5, 'stock', 'flux']


[['stock'], ['flux', 'différent'], 1, 5, ['stock'], ['flux']]


pour le sujet : 
    -> si la position est la même 
    -> s'il n'est pas déjà dedans : 
        -> on ajoute le nouvel objet dans l'ancien objet 


In [15]:
from liste_pattern import pattern_possession
liste_pattern_possession = pattern_possession()
def possession (definition,liste=liste_pattern_possession) : 
    """
    @input : une définition
    @output : les deux éléments liés par une relation de possession lémmatizés 
                indication pour savoir si la définition ne contient pas de relation de possession
    """

    #pattern de type l'ensemble des produits qu'elle conserve
    # [1] que [2] verbe de possession
    pattern_forme_que = liste[0]

    #pattern de type "Le stock correspond à l'ensemble des produits qu'une entreprise possède, stocké en attendant leurs utilisations ou leurs ventes."
    pattern_forme_que_2 = liste[1]

    #même pattern, toutefois dependency matcher ne permets pas l'utilisation de "OP":"?", c'est pourquoi nous devons rajouter un pattern permettant de retrouver les objets ayant un determinant
    pattern_forme_que_avec_determinant = liste[2]
    
    #passif
    #pattern de type : [1] appartient à [2], [1] conservé par [2]
    poss1 = liste[3] #+global
    poss_par = liste[5]#+précis

    #actif
    #pour les pattern de type : [2] possede [1]
    poss2 = liste[4]

    #<objet> est un détenteur de <sujet>
    poss_detenteur = liste[6]

    matcher = DependencyMatcher(nlp.vocab)
    matcher.add("forme_que", [pattern_forme_que])
    matcher.add("forme_que_det",[pattern_forme_que_avec_determinant])

    docu = nlp(definition)
    resultat = []
    preT = pre_traitement(definition,docu)
    docu = preT[0]
    doc = preT[1]
    docc = doc #On crée une copie du doc sous le nom docc
    for token in doc: 
        res =  relation(definition,matcher, doc, 3, 1,0,docu)
        resu = res[1]
        emplacement_verbe = res[0]
        if len(resu)>0 : 
            resultat.append(resu[0])
            doc = doc[emplacement_verbe+1:] #Pour continuer à chercher les autres relations d'équivalence qui se situent après le verbe d'équivalence identifié
        
        else : 
            matcher = DependencyMatcher(nlp.vocab)
            matcher.add("poss_par",[poss_par])
            res = relation(definition,matcher,doc,0,2,1,docu)
            resu = res[1]
            emplacement_verbe = res[0]
            if len(resu)>0:
                resultat.append(resu[0])
                doc = doc[emplacement_verbe+1:]
        
            else:
                matcher = DependencyMatcher(nlp.vocab) 
                matcher.add("poss1_passif",[poss1])
                matcher.add("poss2_actif",[poss2])
                res =  relation(definition,matcher, doc, 2, 1,0,docu)
                resu = res[1]
                emplacement_verbe = res[0]
                if len(resu)>0 : 
                    resultat.append(resu[0])
                    doc = doc[emplacement_verbe+1:] #Pour continuer à chercher les autres relations d'équivalence qui se situent après le verbe d'équivalence identifié
                
            
                    
                else:
                    matcher = DependencyMatcher(nlp.vocab) 
                    matcher.add("pattern_forme_que_2",[pattern_forme_que_2])
                    res =  relation(definition,matcher, doc, 3, 2,1,docu)
                    resu = res[1]
                    emplacement_verbe = res[0]
                    if len(resu)>0 : 
                        resultat.append(resu[0])
                        doc = doc[emplacement_verbe+1:] #Pour continuer à chercher les autres relations d'équivalence qui se situent après le verbe d'équivalence identifié

                    else:
                        matcher = DependencyMatcher(nlp.vocab) 
                        matcher.add("poss_detenteur",[poss_detenteur])
                        res =  relation(definition,matcher, doc, 2, 1,0,docu)
                        resu = res[1]
                        emplacement_verbe = res[0]
                        if len(resu)>0 : 
                            resultat.append(resu[0])
                            doc = doc[emplacement_verbe+1:] #Pour continuer à chercher les autres relations d'équivalence qui se situent après le verbe d'équivalence identifié
                        
                        else :
                            #pour lorsqu'il y a plusieurs relations d'équivalence, après ET avant la relation détectée en premier lieu
                            #exemple : 
                            #text = "Le bilan comptable est un document qui synthétise à un moment le patrimoine d’une entreprise. Ce patrimoine comporte un actif (ses avoirs : terrains, machines, stocks...) et un passif (ses ressources : capital, réserves, crédits.). Le bilan est donc une « photographie » du patrimoine de l’entreprise à un moment donné de son histoire."
                            #[[['patrimoine'], ['actif', 'passif'], 18, 21], [['bilan'], ['photographie', 'patrimoine', 'entreprise'], 29, 34], [['bilan', 'comptable'], ['document'], 1, 5]]
                            if len(resultat)>0 : 
                                j=(resultat[0][2])
                                if j is not None :              
                                    if(emplacement_verbe<j):
                                        if doc.text != docc[:j-1].text:
                                            text = (docc[:j-1]).text
                                            doc = nlp(text)
                                            f = possession(text)
                                            if len(f)>0:
                                                resultat.append(f[0])
                                                break        
        return(resultat)
        
possession("Un actionnaire est un détenteur d'actions d'une société, ce qui fait de lui un copropriétaire de l'entreprise. En tant que copropriétaire, il a le droit de prendre part aux décisions clés de la société, d'obtenir une part des bénéfices sous forme de dividendes et d'avoir accès à une partie de la valeur de l'entreprise en cas de liquidation.")
#[[['action', 'société'], ['actionnaire'], 6, 1, 'actions', 'actionnaire']]


[[['action', 'société'], ['actionnaire'], 6, 1, ['action'], ['actionnaire']]]

In [16]:
def add_cent (add, cent): 
    """
    @input : add :une liste contenant le groupe nominal analysé, cette liste contient également l'élément central
                  chaque élément du groupe nominal est sous forme de lemme
             cent : l'élément central dans la définition, qui n'est pas sous forme de lemme
    @output : la liste add sans l'élément central 
    @exemple : pour [[['action', 'société'], ['actionnaire'], 6, 1, 'actions', 'actionnaire']]
               input : 
                        add :['action', 'société']
                        cent :'actions'
               output : ['société']
    """
    if type(cent) == list : 
        cent = cent[0]
    else :
        cent = cent
    cent_nlp = nlp(cent)

    for token in cent_nlp : 
        sujet_centre = token.lemma_

    sujet_additionnels = add[:]
    sujet_additionnels.remove(sujet_centre)
    return (sujet_additionnels)


In [17]:
from liste_pattern import pattern_but
liste_pattern_but = pattern_but()

def trouver_but (definition,liste=liste_pattern_but):
    """
    @input : une définition
    @output : Si la définition ne contient pas de relation de but, retourne "La définition ne contient pas de relation de but !"
                Si la définition contient une relation de but,  retourne de la relation de but jusqu'à la fin du document
                permet également d'obtenir la variable self.verbe_but qui contient le verbe le plus proche du début de la relation de but 
                afin de pouvoir ensuite retrouver le sujet de la relation de but à l'aide de la fonction sujet_du_but

    @exemples : 
        input : "durée de l’ensemble des procédés permettant à une entreprise réaliser des bénéfices.(fournisseurs, matières premières,investissement,etc…)"
        output :'réaliser des bénéfices.(fournisseurs , matières premières , investissement ,'


        input : "Un stock est un ensemble de fonds monétaires investis par l’entreprise dans le but de l’obtention de machines."
        output :'dans le but de l’ obtention de machines' 
                   
    """
    doc = nlp(definition)
    ok = []
    pattern_t_1 = [
        {'LEMMA': {"IN": ['objectif','but','finalité','rôle','mission']},'POS': 'NOUN'} ,{'DEP': {"IN": ['acl','amod']},'OP':'?'}, {'LEMMA': {"IN": ['être','consiste','consister']}}]
    #up + amod + rôle
    matcher_t = Matcher(nlp.vocab)     
    matcher_t.add("but_descriptif", [pattern_t_1])

    matches = matcher_t(doc)
    if (len(matches) != 0) :
        for match_id, start, end in matches:
            ok= [t.text for t in doc[end-1].subtree]
            if len(ok)>1:
                ok = (' '.join(ok))
                return(ok)
        
    pattern_t_2 = [{'LEMMA':'dans'},{'POS':'DET'},{'LEMMA': {"IN": ['objectif','but','finalité','rôle','mission']},'POS': 'NOUN'} ,{'DEP': {"IN": ['acl','amod']},'OP':'?'}
    ]
    #dans le but de
    matcher_t = Matcher(nlp.vocab)     
    matcher_t.add("but_descriptif_danslebutde", [pattern_t_2])
    matches = matcher_t(doc)
    if (len(matches) != 0) :
        for match_id, start, end in matches:
            ok= [t.text for t in doc[end-1].subtree]
            ok = (' '.join(ok))
            return(ok)
    

    pattern_t_3 = [{'LEMMA': 'être'},{'POS': 'DET'},{'LEMMA': {"IN": ['objectif','but','finalité','rôle','mission']},'POS': 'NOUN'}]
    matcher_t.add("but_passif", [pattern_t_3])  
    matches = matcher_t(doc)
    if (len(matches) != 0) :
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]
            if doc[end-1].dep_ != "obj":
                ok= [t.text for t in doc[end-1].subtree]
                ok = (' '.join(ok))
                return(ok)
            elif doc[end-1].dep_ == "obj" and doc[end-1].text in ["but","objectif","finalité",'rôle']:
                pos1 = doc[end-1].i
                pos = doc[end-1].head.i
                return doc[pos:pos1+1]
    pattern_t_4 = [
    {'LEMMA':"avoir","POS":{"IN":["AUX","VERB"]}},  {'POS': {"IN": ["ADP","ADV"]},'OP':'?'},{'POS': "ADP",'OP':'?','LEMMA':{'IN':['comme','pour']}},{'DEP': {"IN": ['acl','amod']},'OP':'?'},
    {'LEMMA': {"IN": ['objectif','but','finalité','mission','rôle']},'POS': 'NOUN'} ,{'DEP': {"IN": ['acl','amod']},'OP':'?'},
    {'LEMMA': {"IN": ['d','de']},"OP":"?"}]

    matcher_t = Matcher(nlp.vocab)     
    matcher_t.add("but_descriptif2", [pattern_t_4])
    matches = matcher_t(doc)
    if (len(matches) != 0) :
        for match_id, start, end in matches:
            ok= [t.text for t in doc[end+1].subtree]
            ok = (' '.join(ok))
            return(ok)


    patterns_d_1 = liste[0]

    patterns_d_2 = liste[1]

    patterns_d_3 = liste[2]

    patterns_d_4 = liste[3]

    patterns_d_5 = liste[4]

    
    matcher.add("V+xcomp", [patterns_d_1])
    matcher.add("permission", [patterns_d_2])

    matches = matcher(doc)
    if (len(matches) != 0) :
        match_id, token_ids = matches[0]
        ok= [t.text for t in doc[token_ids[1]+1].subtree]
        empty_word = ["en","attendant","au","vue","attente","En","prévision"]
        if len(ok)>3:
            for i in range (2) :
                if len(ok)>0 : 
                    if ok[0] in empty_word:
                        ok.pop(0)
                        pass
            ok = (' '.join(ok))
            return(ok)
        else : 
            matcher.add("but_xcomp", [patterns_d_3])
            if (len(matches) != 0) :
                match_id, token_ids = matches[0]
                ok= [t.text for t in doc[token_ids[0]].subtree]
                if ok[0] == "pour":
                    ok = (' '.join(ok))
                    return(ok)
                
            ok= [t.text for t in doc[token_ids[1]].subtree]
            if len(ok)>3:
                for i in range (2) :
                    if len(ok)>0 : 
                        if ok[0] in empty_word:
                            ok.pop(0)
                            pass
                ok = (' '.join(ok))
                return(ok)
            else : 
                el = (doc[token_ids[1]].text)
                sub_elem = [(t.text) for t in doc]
                ok = (sub_elem[sub_elem.index(el)::])
                ok = (' '.join(ok))
                return(ok)
    
    
    matcher.add("AUVUE2", [patterns_d_4])
    matcher.add("an_attendant", [patterns_d_5])
    matches = matcher(doc)
    if (len(matches) != 0) :
        match_id, token_ids = matches[0]
        el = (doc[token_ids[1]].text)
        sub_elem = [(t.text) for t in doc]
        ok = (sub_elem[sub_elem.index(el)+2::])
        ok = (' '.join(ok))
        return(ok)
    else : 
        return False
    


In [18]:
#Fonction pour voir s'il y a plusieurs relations de but séparés par un 'ou' ou un 'et'
def plusieurs_buts (relation_de_but):
    """
    @input : la relation de but retrouvée par la fonction but
    @output : une liste contenant les différents buts s'il y en a 
                Faux s'il n'y a qu'une seule relation de but

    @exemples : 
            input : à trouver des capitaux pour la financer ou la vendre
            output : ['à trouver des capitaux pour la financer', 'ou la vendre']            
    """
    liste_des_buts = []
    doc = nlp(relation_de_but)
    for token in doc : 
        if token.text == 'ou':
            index_depart = relation_de_but.index(' ou ')
            premier_element = relation_de_but[:index_depart]
            deuxieme_element = relation_de_but[index_depart+1:]
            liste_des_buts.append(premier_element)
            liste_des_buts.append(deuxieme_element)
            break
        elif token.text == 'et':
            index_depart = relation_de_but.index(' et ')
            premier_element = relation_de_but[:index_depart]
            deuxieme_element = relation_de_but[index_depart+1:]
            liste_des_buts.append(premier_element)
            liste_des_buts.append(deuxieme_element)
            break
    if len(liste_des_buts) == 0 :
        
        return ([relation_de_but])
    else : 
        for ell in liste_des_buts: 
            el = nlp(ell)
            for token in el : 
                if token.lemma_ == 'contrepartie':
                    liste_des_buts.remove(ell)
        return (liste_des_buts)
    

In [19]:

"""
@input : le dictionnaire
@output : 
transformation de l'entrée objet du but du dictionnaire de : 
["en attente d' utilisation", 'ou de vente .', 'pour être vendus']
à
['utilisation', 'vente', 'vendre']
"""
def decomposition_but_dictionnaire(dictionnaire):
    i = dictionnaire[2]['BUT'][0]['O']
    for j in range (len(i)) :
        el = (i[j])
        doc = nlp(el)
        for token in doc : 
            if (token.pos_ == "NOUN"or token.pos_ == "VERB" or token.pos_ == "ADJ" or token.lemma_ == "sans" ) and len(token)>1: 
                i[j]=(token.lemma_)
    return(dictionnaire)

def decomposition_but_liste(liste):
    """
    exemple : 
    liste = ['en attendant leurs utilisations', 'ou leurs ventes . "']
    retour = ['attendre', 'utilisation', 'ou','vente']
    """
    listee = ["sans","ou","et","pas"]
    liste2 = []
    for j in range (len(liste)) :
        el = (liste[j])
        doc = nlp(el)
        for token in doc : 

            if (token.pos_ == "NOUN"or token.pos_ == "VERB" or token.pos_ == "ADJ" or token.lemma_ in listee  ) and len(token)>1: 
                liste2.append(token.lemma_)
    return(liste2)

Visualisation des relations dans les définitions : 

In [20]:
def fichier (document,nouveau_corpus) : 
    """
    @input : un document sous forme txt
    @output : un nouveau document txt indiquant la présence des différentes relations ainsi que les éléments interagissant 
             dans celles-ci
    """
    j=0
    with open (document, "r", encoding = 'utf-8') as fichier : 
        with open(nouveau_corpus, "w", encoding= 'utf-8') as pli :
            for lignes in fichier:
                
                lines = [lignes]
                lines[-1] = lines[-1].rstrip('\n')
                for line in lines : 
                    pli.write(f"\n'{j}'"+line+"\n")
                    j+=1
                    
                    #equivalence
                    equivalence_resultat = equivalence(line)
                    if len(equivalence_resultat) != 0 :
                        for i in range (len(equivalence_resultat)):
                        
                            sujet_centre = equivalence_resultat[i][4]
                            sujet_additionnel = equivalence_resultat[i][0]
                            suj_add = add_cent(sujet_additionnel,sujet_centre)

                            objet_centre = equivalence_resultat[i][5]
                            objet_additionnel = equivalence_resultat[i][1]
                            obj_add = add_cent(objet_additionnel,objet_centre)


                            pli.write (f"EQUIVALENCE/ S_centre: '{sujet_centre}'   S_additionnel: '{suj_add}' \nO_centre: '{objet_centre}'   O_additionnel: '{obj_add}'\n")
                            
                    else : 
                        pli.write("La définition ne contient pas de relation d'équivalence' ! \n")
                    
                    
                    #possession
                    possession_resultat = possession(line)
                    if len(possession_resultat) != 0 : 
                        for i in range (len(possession_resultat)):

                            sujet_centre = possession_resultat[i][4]
                            sujet_additionnel = possession_resultat[i][0]
                            suj_add = add_cent(sujet_additionnel,sujet_centre)

                            objet_centre = possession_resultat[i][5]
                            objet_additionnel = possession_resultat[i][1]
                            obj_add = add_cent(objet_additionnel,objet_centre)


                            pli.write (f"POSSESSION/ S_centre: '{sujet_centre}'   S_additionnel: '{suj_add}' \n O_centre: '{objet_centre}'   O_additionnel: '{obj_add}'\n")

                    else : 
                        pli.write("La définition ne contient pas de relation de possession' ! \n")
                    
                    #but
                    but = trouver_but(line)
                    if but != False : 
                        resultat = plusieurs_buts(but)
                        liste_objet = resultat
                        pli.write (f"Le but est'{liste_objet}' \n")
                        liste_objet_decompose = decomposition_but_liste(liste_objet)
                        pli.write (f"Le but est'{liste_objet_decompose}' \n")
                    else : 
                        pli.write("La définition ne contient pas de relation de but ! \n")


#fichier("stock_prof.txt","stock_profRELATIONS.txt")


In [21]:
def retour_complet (definition) : 
    """
    @input : un document sous forme txt
    @output : un nouveau document txt indiquant la présence des différentes relations ainsi que les éléments interagissant 
             dans celles-ci
    """
    line = definition
    equivalence_resultat = equivalence(line)
    if len(equivalence_resultat) != 0 :
        for i in range (len(equivalence_resultat)):
        
            liste_sujet = equivalence_resultat[i][0]
            liste_objet = equivalence_resultat[i][1]
            print (f"equivalence : sujet: '{liste_sujet}' \t objet: '{liste_objet}' \n")
            
    else : 
        print("La définition ne contient pas de relation d'équivalence' ! \n")
    
    
    #possession
    possession_resultat = possession(line)
    if len(possession_resultat) != 0 : 
        for i in range (len(possession_resultat)):
            liste_sujet = possession_resultat[i][0]
            liste_objet = possession_resultat[i][1]
            print (f"possession : sujet: '{liste_sujet}' \t objet: '{liste_objet}' \n")
    else : 
        print("La définition ne contient pas de relation de possession' ! \n")
    
    #but
    but = trouver_but(line)
    if but != False : 
        resultat = plusieurs_buts(but)
        liste_objet = resultat
        print (f"Le but est'{liste_objet}' \n")
        liste_objet_decompose = decomposition_but_liste(liste_objet)
        print (f"Le but est'{liste_objet_decompose}' \n")
    else : 
        print("La définition ne contient pas de relation de but ! \n")

#retour_complet("C’est le nombre de biens que l’on possède, qui ne sont pas encore vendus ou consommés")

equivalence : sujet: '['’']' 	 objet: '['nombre', 'bien']' 

possession : sujet: '['nombre', 'bien']' 	 objet: '['’']' 

La définition ne contient pas de relation de but ! 



In [ ]:
def fichiereq (document,nouveau_corpus) : 
    """
    @input : un document sous forme txt
    @output : un document sous forme txt indiquant la présence de la relation d'équivalence
    """
    j=0
    with open (document, "r", encoding = 'utf-8') as fichier : 
        with open(nouveau_corpus, "w", encoding= 'utf-8') as pli :
            for lignes in fichier:
                
                lines = [lignes]
                lines[-1] = lines[-1].rstrip('\n')
                for line in lines : 
                    pli.write(f"\n'{j}'"+line+"\n")
                    j+=1
                    
                    #equivalence
                    equivalence_resultat = equivalence(line)
                    if len(equivalence_resultat) != 0 :
                        for i in range (len(equivalence_resultat)):
                        
                            liste_sujet = equivalence_resultat[i][0]
                            liste_objet = equivalence_resultat[i][1]
                            pli.write (f"S: '{liste_sujet}' \O: '{liste_objet}' \n")
                            
                    else : 
                        pli.write("La définition ne contient pas de relation d'équivalence' ! \n")



def fichierposs (document,nouveau_corpus) : 
    """
    @input : un document sous forme txt
    @output : un document sous forme txt indiquant la présence de la relation de possession
    """
    j=0
    with open (document, "r", encoding = 'utf-8') as fichier : 
        with open(nouveau_corpus, "w", encoding= 'utf-8') as pli :
            for lignes in fichier:
                
                lines = [lignes]
                lines[-1] = lines[-1].rstrip('\n')
                for line in lines : 
                    pli.write(f"\n'{j}'"+line+"\n")
                    j+=1
                    
                    #possession
                    possession_resultat = possession(line)
                    if len(possession_resultat) != 0 : 
                        for i in range (len(possession_resultat)):
                            liste_sujet = possession_resultat[i][0]
                            liste_objet = possession_resultat[i][1]
                            pli.write (f"sujet '{liste_sujet}' \t objet '{liste_objet}' \n")
                    else : 
                        pli.write("La définition ne contient pas de relation de possession' ! \n")


def fichierbut (document,nouveau_corpus) : 
    """
    @input : un document sous forme txt
    @output : un document sous forme txt indiquant la présence de la relation de but
    """
    j=0
    with open (document, "r", encoding = 'utf-8') as fichier : 
        with open(nouveau_corpus, "w", encoding= 'utf-8') as pli :
            for lignes in fichier:
                
                lines = [lignes]
                lines[-1] = lines[-1].rstrip('\n')
                for line in lines : 
                    pli.write(f"\n'{j}'"+line+"\n")
                    j+=1
                    
                    #but
                    but = trouver_but(line)
                    if but != False : 
                        resultat = plusieurs_buts(but)
                        liste_objet = resultat
                        pli.write (f"Le but est'{liste_objet}' \n")
                        liste_objet_decompose = decomposition_but_liste(liste_objet)
                        pli.write (f"Le but est'{liste_objet_decompose}' \n")
                    else : 
                        pli.write("La définition ne contient pas de relation de but ! \n")


In [23]:
from liste_synonymes import dico_syno
dictionnaire_synonymes = dico_syno()
def liste_en_synonyme (dictionnaire_synonymes,liste_recue):
    """
    @input : Un dictionnaire contenant des listes de synonymes
             Une liste de mots 
    @output : Pour chaque mot présent dans la liste reçue, retourne une liste de listes de synonymes pour ces mots 
    """
    mot_important_synonymes = []
    mot_pas_dans_synonymes =[]
    for i in range(len(liste_recue)):
        terme = liste_recue[i]
        for liste_synonyme in dictionnaire_synonymes :
            if terme in liste_synonyme and liste_synonyme not in mot_important_synonymes:
                mot_important_synonymes.append(liste_synonyme)
                mot_pas_dans_synonymes.append(terme)

        if any(terme in sublist for sublist in mot_important_synonymes) == False : 
            mot_important_synonymes.append([terme])
    
    return(mot_important_synonymes)

Correction

In [24]:
#dictionnaire de définitions correctes du stock
dico = {"stock":["Le stock correspond à l'ensemble des produits qu'une entreprise possède, stocké en attendant leurs utilisations ou leurs ventes."
,"Le stock représente tous les produits possédés par une entreprise qui sont stockés en attente d'utilisation ou de vente."
,"Un stock est un ensemble de produits physiques (marchandises, matières premières ou produits finis) possédés par une entreprise, en attendant d'être utilisés dans le processus de production ou d'être vendus sur un marché."
,"Un stock est un agrégat qui intègre à la période présente des éléments qui étaient déjà présents dans l’agrégat de la période précédente . La notion de stock s’oppose ainsi à celle de flux, qui est un agrégat qui se renouvelle entièrement de période en période . Donc un flux est remis à 0 au début de chaque période, alors qu’un stock a un « effet mémoire » . De manière imagée, on peut dire que le stock est un réservoir d’eau, tandis que le flux correspond à la quantité d’eau ajoutée (flux entrant) ou enlevée (flux sortant) à chaque période . Le stock est donc une accumulation de flux entrants, moins une accumulation de flux sortants"
,"Le stock d'une entreprise représente l'ensemble des produits qu'elle conserve en vue de leur utilisation ou de leur vente."
,"L'entreprise possède un stock de produits qui est stocké en vue de leur utilisation ou de leur vente."
,"Le stock d'une entreprise est constitué de tous les produits qu'elle possède et qu'elle stocke en vue de leur utilisation ou de leur vente."
,"En attendant d'être utilisés ou vendus, les produits possédés par une entreprise sont stockés dans son stock."
,"Tous les produits qu'une entreprise possède et qu'elle stocke en vue de leur utilisation ou de leur vente sont appelés le stock de l'entreprise."
,"Le stock correspond à l'ensemble des articles détenus par une entreprise et qui sont stockés en vue d'être utilisés ou vendus."
,"Le stock désigne tous les produits détenus par une entreprise qui sont stockés en attente d'être utilisés ou vendus."
,"Le stock représente l'ensemble des articles possédés par une entreprise qui sont stockés en prévision de leur utilisation ou de leur vente."
,"Le stock englobe tous les produits détenus par une entreprise et qui sont stockés en attente d'utilisation ou de vente."
,"Le stock désigne tous les produits possédés par une entreprise qui sont stockés en vue d'être utilisés ou vendus."
]}

In [25]:
def nom_et_verbe (definition):
    """
    @input : une définition
    @output : les noms et verbes (sous formes de lemmes) de la définition dans une liste
    exemple : 
        definition = "Le stock correspond à l'ensemble des produits qu'une entreprise possède, stocké en attendant leurs utilisations ou leurs ventes."
        nom_et_verbe(definition)
        RETOUR : ['stock','correspondre','ensemble','produit','entreprise','posséder','stocker','attendre','utilisation','vente']
    """
    l = []
    doc = nlp(definition)
    for token in doc : 
        if token.pos_ == "VERB" or token.pos_ == "NOUN" or token.pos_ =="ADJ":
            l.append(token.lemma_)
    return l

In [26]:
matcher = DependencyMatcher(nlp.vocab) 
def exemple (definition):
    matcher = DependencyMatcher(nlp.vocab) 

    par_exemple = [
    {"RIGHT_ID": "anchor_founded","RIGHT_ATTRS": {"POS":"VERB"}},
    {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "exemple","RIGHT_ATTRS": {"DEP": "obl:mod","LEMMA":{"IN": ['exemple','ex']}}},
    {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "fin","RIGHT_ATTRS": {"DEP": "obj"}}
   ]
    
    
    par_exemple_2 = [
    {"RIGHT_ID": "anchor_founded","RIGHT_ATTRS": {"POS":"NOUN"}},
    {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "exemple","RIGHT_ATTRS": {"DEP": "nmod","LEMMA":{"IN": ['exemple','ex']}}},
    {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "fin","RIGHT_ATTRS": {"DEP": {"IN": ['nmod','amod']},"LEMMA":{"NOT_IN": ['exemple','ex']}}}
   ]
    
    par_exemple_3 = [
    {"RIGHT_ID": "anchor_founded","RIGHT_ATTRS": {"POS":"NOUN"}},
    {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "exemple","RIGHT_ATTRS": {"DEP": "nmod","LEMMA":{"IN": ['exemple','ex']}}}
    
   ]

    #text = "le cycle d’investissement est le plus souvent utilisé pour des opérations sur le long terme il sert à l'acquisition d’immobilisation. Il peut s'agir d'un achat obligatoire par exemple"
    par_exemple_4 = [
    {"RIGHT_ID": "noun","RIGHT_ATTRS": {"POS":"NOUN"}},
    {"LEFT_ID": "noun","REL_OP": ">","RIGHT_ID": "anchor_founded","RIGHT_ATTRS": {"POS": "ADJ","DEP":"acl"}},
    {"LEFT_ID": "anchor_founded","REL_OP": ">","RIGHT_ID": "exemple","RIGHT_ATTRS": {"DEP": "obl:mod","LEMMA":{"IN": ['exemple','ex']}}},
    
   ]
    doc = nlp(definition)
    matcher.add("par_exemple", [par_exemple])
    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        sujet = doc[token_ids[0]+1:token_ids[2]+1]
        return (sujet)

    matcher.add("par_exemple_2", [par_exemple_2])
    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        if token_ids[0] < token_ids[1] : 
            sujet = doc[token_ids[0]:token_ids[1]+1] #achats de machines par exemple
        else : 
            sujet = doc[token_ids[1]-1:token_ids[2]+1] #par exemple l'achats

        return (sujet)
    matcher.add("par_exemple_3", [par_exemple_3])
    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        if token_ids[0] < token_ids[1] : 
            sujet = doc[token_ids[0]:token_ids[1]+1] #achats de machines par exemple
        else : 
            sujet = doc[token_ids[1]-1:token_ids[0]+1] #par exemple l'achats
        return (sujet)

    matcher.add("par_exemple_4", [par_exemple_4])
    matches = matcher(doc)
    if len(matches)!=0 :
        match_id, token_ids = matches[0]
        if token_ids[0] < token_ids[1] : 
            sujet = doc[token_ids[0]:token_ids[2]+1] #achats de machines par exemple
        else : 
            sujet = doc[token_ids[2]-1:token_ids[0]+1] #par exemple l'achats
        return (sujet)

    else : 
        return False
    
"""
par exemple aux intérêts payés sur ses dettes bancaires, les remboursements
achat obligatoire par exemple
par exemple l'achat obligatoire
Par exemple pour une entreprise commerciale
"""

def exemple_2 (definition):
    """
    @input : une définition
    @output : les exemples entre parenthèse, signalés comme : 
            ("par exemple : ...")
            ("... par exemple")
    """
    #Exemples entre parenthèse
    doc = nlp(definition)       
    #parenthèse avec exemple en début ou en fin de parenthèse
    debut = False
    fin = False
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ == "(" : 
                if doc[token.i+1].text in ["ex","exemple"] or doc[token.i+1].text == "par" and doc[token.i+2].text in ["ex","exemple"] :
                    debut = token.i
                    break
            if token.lemma_ == ")":
                fin = token.i 
    if debut == False :
        for sent in doc.sents:
            for token in sent:
                if token.lemma_ == "(" : 
                        debut = token.i
                        break
                
                if token.lemma_ == ")":
                    if doc[token.i-2].text == "par" and doc[token.i-1].text in ["ex","exemple"] :
                        fin = token.i 
    if fin != False or debut != False : 
        return (doc[debut:fin+1])
    else : 
        return False

def retour_exemple (definition): 
    retour = exemple(definition)
    if type(retour) != bool : 
        return retour
    retour = exemple_2(definition)
    if type(retour) != bool:
        return retour
    else : 
        return False 

In [32]:
def representation_liste_dictionnaire_e(terme,definition):
    
    """
    @input : une définition
    @output : les relations et termes qui interagissent sous forme de liste de dictionnaire
            sous la forme : 
            {'EQUIVALENCE': {'S': ['sujet_e1', 'sujet_e2', 'sujet_e3'], 'O': ['objet_e1', 'objet_e2', 'objet_e3']}, 
            'POSSESSION': {'S': ['sujet_p1', 'sujet_p2', 'sujet_p3'], 'O': ['objet_p1', 'objet_p2', 'objet_p3']}, 
            'BUT': {'S': ['sujet_b1', 'sujet_b2', 'sujet_b3'], 'O': ['objet_b1', 'objet_b2', 'objet_b3']}}

    @exemple : 
        input : "Le cycle de financement regroupe toutes les opérations financières qui concernent l'activité d'une entreprise. On peut le considérer comme la contrepartie du cycle d'investissement et d'exploitation, car il sert à trouver des capitaux pour les financer."
        output : {'EQUIVALENCE': {'S': ['financement'], 'O': ['activité', 'entreprise']}, 'BUT': {'S': ['cycle de financement'],'O': ['trouver', 'capital', 'financer']}}

    """
    test_terme = {}
    #equivalence
    equivalence_resultat = equivalence(definition)
    s_o = {}
    if len(equivalence_resultat) != 0 :
        for i in range (len(equivalence_resultat)):
            #print('ici',equivalence_resultat[i])
            #[['stock'], ['groupe', 'bien', 'matière'], 1, 4]
            
            liste_sujet = equivalence_resultat[i][0]
            position_sujet = equivalence_resultat[i][0]
            liste_objet = equivalence_resultat[i][1]
            s_o["S"]=liste_sujet
            s_o["O"]=liste_objet
            test_terme["EQUIVALENCE"]=s_o

    #possession
    possession_resultat = possession(definition)
    s_o = {}
    if len(possession_resultat) != 0 : 
        for i in range (len(possession_resultat)):
            liste_sujet = possession_resultat[i][0]
            liste_objet = possession_resultat[i][1]
            s_o["S"]=liste_sujet
            s_o["O"]=liste_objet
            test_terme["POSSESSION"]=s_o


    #but
    but_resultat = trouver_but(definition)
    if but_resultat != False :
        resultat = plusieurs_buts(but_resultat)
        s_o = {}
        #liste_sujet = self.sujet_du_but(definition)
        s_o["S"]=[terme]
        liste_objet = resultat #['vendus', 'ou acheté']
        liste_objet = decomposition_but_liste(liste_objet) #['vendre', 'acheter']
        s_o["O"]=liste_objet
        test_terme["BUT"]=s_o

    return(test_terme)

def ajout_definition(definition,dictionnaire,terme = None):
    
    #equivalence
    equivalence_resultat = equivalence(definition)
    if len(equivalence_resultat) != 0 :
        for i in range (len(equivalence_resultat)):
            liste_sujet = equivalence_resultat[i][0]
            liste_objet = equivalence_resultat[i][1]
            
        es = dictionnaire['EQUIVALENCE']['S']
        eo = dictionnaire['EQUIVALENCE']['O']

        if "EQUIVALENCE" in dictionnaire : 
            terme = dictionnaire['EQUIVALENCE']['S']
            for i in liste_sujet : 
                if i in es :
                    #Pour ajouter au dictionnaire uniquement les relations d'équivalence qui portent bien sur le même sujet
                    #stock marchand pour stock accepté
                    for i in liste_objet : 
                        if i not in eo :
                            eo.append(i)
        else : 
            s_o["S"]=liste_sujet
            s_o["O"]=liste_objet
            dictionnaire["EQUIVALENCE"]=s_o
        

    #possession
    possession_resultat = possession(definition)
    if len(possession_resultat) != 0 : 
        for i in range (len(possession_resultat)):
            liste_sujet = possession_resultat[i][0]
            liste_objet = possession_resultat[i][1]

        if "POSSESSION" in dictionnaire : 
            ps = dictionnaire['POSSESSION']['S']
            for i in liste_sujet :
                if i not in ps :
                    ps.append(i)
            
            po = dictionnaire['POSSESSION']['O']
            for i in liste_objet :
                if i not in po :
                    po.append(i)
        else : 
            s_o["S"]=liste_sujet
            s_o["O"]=liste_objet
            dictionnaire["POSSESSION"]=s_o

    #but
    but = trouver_but(definition)
    if but != False :
        if 'BUT' in dictionnaire : 
            b = dictionnaire['BUT']['O']
            resultat = plusieurs_buts(but)
            liste_objet = resultat
            liste_objet = decomposition_but_liste(liste_objet)
            for i in liste_objet:
                if i not in b :
                    b.append(i)
        else : 
            #on reprend le dictionnaire et y ajoute le but :

            resultat = plusieurs_buts(but)
            s_o = {}
            s_o["S"]=terme
            liste_objet = resultat #['vendus', 'ou acheté']
            liste_objet = decomposition_but_liste(liste_objet) #['vendre', 'acheter']
            s_o["O"]=liste_objet
            dictionnaire["BUT"]=s_o

    return (dictionnaire)

def dicot(dico):
    """
    @input :
    un dictionnaire avec comme clef le terme définit et comme valeur une liste de définitions correctes du terme
    @output : 
    le terme analysé sous forme de dictionnaire
    """
    keys = list(dico.keys())
    terme = keys[0]
    def1 = dico[terme][0]
    f = representation_liste_dictionnaire_e(terme,def1)
    for i in range (1,len(dico[terme])):
        definition = (dico[terme][i])
        f = ajout_definition(definition,f)
    return(f,terme)


def construire_representation_dictionnaire(dictionnaire_synonymes, definitions_professeur):
    """
    @input : une liste de définitions du professeur d'un terme en particulier 
             un dictionnaire de synonymes
    @output : retourne les définitions du terme défini par le professeur sous forme de dictionnaire 
    @durée : +-9s
    @exemple : input: le dictionnaire "dico" de définitions de stock
             : output : ({'EQUIVALENCE': {'S': ['stock'], 'O': [['ensemble', 'réserve', 'quantité', 'nombre', 'accumulation','agrégat'], ['produit', 'article', 'élément', 'bien', 'actif', 'avoir', 'ressource'], ['physique']]}, 
                        'POSSESSION': {'S': ['ensemble', 'produit', 'physique', 'stock', 'article'], 'O': [['entreprise', 'organisation', ['organisme', 'activité', 'marchand']]]}, 
                        'BUT': {'S': 'stock', 'O': [['utilisation', 'utiliser'], ['ou'], ['vente', 'vendre', 'revendre', 'revente', 'commercialisation', 'cession'], ['processus', 'fonctionnement', ['pilotage', 'quotidien']], ['produire', 'production'], ['marché']]}}, 
                        'stock')
    """
    d1 = dicot(definitions_professeur)
    d = d1[0]
    terme = d1[1]
    for i in list(d.keys()):
        objets_dans_dico = (d[i]["O"])
        motmot = (dictionnaire_synonymes['mots'])
        d[i]["O"] = liste_en_synonyme(motmot,objets_dans_dico)
    return(d,terme)

construire_representation_dictionnaire(dictionnaire_synonymes, dico)




({'EQUIVALENCE': {'S': ['stock'],
   'O': [['ensemble',
     'réserve',
     'quantité',
     'nombre',
     'accumulation',
     'agrégat'],
    ['produit',
     'article',
     'élément',
     'bien',
     'actif',
     'avoir',
     'ressource',
     ['élément', 'physique'],
     ['bien', 'physique']],
    ['physique']]},
  'POSSESSION': {'S': ['ensemble', 'produit', 'physique', 'stock', 'article'],
   'O': [['entreprise',
     'organisation',
     ['organisme', 'activité', 'marchand']]]},
  'BUT': {'S': ['stock'],
   'O': [['utilisation', 'utiliser'],
    ['ou'],
    ['vente', 'vendre', 'revendre', 'revente', 'commercialisation', 'cession'],
    ['processus', 'fonctionnement', ['pilotage', 'quotidien']],
    ['produire', 'production'],
    ['marché']]}},
 'stock')

Comparaison Schémas conceptuels

In [ ]:

"""
@input : schéma conceptuel de l'étudiant et schéma conceptuel correct
@output : Le nombre de relation qui ne correspondent pas dans les deux schémas conceptuels
          Le nombre d'éléments qui ne correspondent pas dans les schémas conceptuels
"""
def nbre_difference (liste_SC, liste_SE):
    mots_SC = []
    mots_SE = []
    
    #différence de nb de relation
    nb_rel = 0
    resu_mots = 0
    nbre_mots_manq = []

    relation_SC = list(liste_SC.keys())
    relation_SE = list(liste_SE.keys())
    for i in relation_SC: 
        if i != "EQUIVALENCE":
            if i not in relation_SE: 
                nb_rel += 1

            if i in relation_SE : 
                mots_SC=liste_SC[i]['O']
                mots_SE=liste_SE[i]['O']
                nbre_mots_manq = mots_SC[:]

                for elemSE in mots_SE :
                    for elemSC in mots_SC: 
                        if elemSE == elemSC or elemSE in elemSC : 
                            nbre_mots_manq.remove(elemSC)
            resu_mots += len(nbre_mots_manq)

    return(nb_rel,resu_mots)

"""
@input :terme définit , liste de définitions correctes, définition de l'étudiant 
@output : le schéma conceptuel correct le plus proche de celui de l'étudiant
durée = 9.7sec
"""
def trouver_SC (terme,liste_def_prof,def_e):
    liste_SC = []
    liste_SC2 = []

    resultat = 0
    nv_resultat = False

    SC_leplusproche = False

    for definition in liste_def_prof: 
        liste_SC.append(representation_liste_dictionnaire_e("stock",definition))

    for j in liste_SC : 
        for i in list(j.keys()):
            objets_dans_dico = (j[i]["O"])
            motmot = (dictionnaire_synonymes['mots'])
            j[i]["O"] = liste_en_synonyme(motmot,objets_dans_dico)
            liste_SC2.append(j)
    
    liste_SE = representation_liste_dictionnaire_e(terme,def_e)

    numero = 0
    bon_numero =0

    for i in liste_SC:
        resultat = nbre_difference(i,liste_SE)
        if type(nv_resultat) == bool:
            nv_resultat=resultat
            SC_leplusproche=i

        if resultat[0] == nv_resultat[0]:
            if resultat[1] <= nv_resultat[1]:
                pass
            if resultat[1] > nv_resultat[1]:
                nv_resultat = resultat
                SC_leplusproche=i
                bon_numero = numero

        if resultat[0] < nv_resultat[0]:
            nv_resultat = resultat
            SC_leplusproche=i
            bon_numero = numero
        
        numero +=1

    return(SC_leplusproche,liste_def_prof[bon_numero])


In [39]:
def construire_representation_dictionnaire_(dictionnaire_synonymes, SC_leplusproche):
    """
    @input : une liste de définitions du professeur d'un terme en particulier 
             un dictionnaire de synonymes
    @output : retourne les définitions du terme défini par le professeur sous forme de dictionnaire 
    """
    for i in list(SC_leplusproche.keys()):
        objets_dans_dico = (SC_leplusproche[i]["O"])
        motmot = (dictionnaire_synonymes['mots'])
        SC_leplusproche[i]["O"] = liste_en_synonyme(motmot,objets_dans_dico)
    return(SC_leplusproche)

"""
Correction avec SC
@Input : SC_leplusproche : Le schéma conceptuel correct le plus proche du schéma conceptuel de l'élève à corriger
         SCacorriger : Le schéma conceptuel de l'élève à corriger
         def_e : Définition de l'élève 
@Output : Correction de la définition de l'élève (sans exemple / antonyme)
@Durée :
"""
def correction_SC (SC_leplusproche,SCacorriger,definition_e,relation_dico="EQUIVALENCE",relation_txt="d'équivalence",terme="stock"):
    retour2= []#nouveau

    RESULTAT = []
    if relation_dico=="EQUIVALENCE":
        p_equivalence = True
    else : 
        p_equivalence = False

    if relation_dico=="POSSESSION":
        p_possession = True
    else : 
        p_possession = False
    
    if relation_dico=="BUT":
        p_but = True
    else : 
        p_but = False

    for cle in SC_leplusproche.keys():
        if cle == relation_dico: 
            eq_p_important = (SC_leplusproche[relation_dico]["O"])
            eq_p_importantS = (SC_leplusproche[relation_dico]["S"])
            RESULTAT.append(f"La relation '{relation_txt}' dans la définition du professeur est S: '{eq_p_importantS}' O:'{eq_p_important}'")
            RESULTAT.append("\n")


            break

        else : 
            eq_p_important=[]

    if eq_p_important == []:
        return (eq_p_important)

    eq_e = []
    for cle in SCacorriger.keys():
        if cle == relation_dico: 
            eq_e = (SCacorriger[relation_dico]['O'])
            eq_eS = (SCacorriger[relation_dico]['S'])
            RESULTAT.append(f"La relation '{relation_txt}' dans la définition de l'élève est : S:'{eq_eS}' O:'{eq_e}' \n")
            RESULTAT.append("\n")
            RESULTAT.append(f"La relation '{relation_txt}' est bien présente")
            break
    else : eq_e = []

    manque = []
    termes_corrects = []
    #première possibilité
    if len(eq_p_important)>0 and len(eq_e)>0 :
        # Vérifier chaque sous-liste de L
        for sous_liste in eq_p_important:
            element_trouve = False
            
            # Vérifier chaque élément de la sous-liste
            for element in sous_liste:
                if type(element) != list : 
                    #soit il s'agit d'une liste de mots tels que 
                    if element in eq_e: #soit il s'agit de synonymes en un seul mot tels que entreprise et organisation dans : ['entreprise', 'organisation',['organisme', 'activité', 'marchand']]
                        element_trouve = True
                        if element not in termes_corrects :
                            termes_corrects.append(element)
                        break
                else : 
                    element_trouvee = True
                    #soit en plusieurs comme organisme qui a une activité marchande dans : ['entreprise', 'organisation',['organisme', 'activité', 'marchand']]
                    #dans ce cas, chacun des mots de cette liste doivent être présent dans la liste d'objet retrouvée de l'élève
                    #On a plusieurs cas de figures, prenons l'exemple 
                    #A : L'élève a correctement retrouvé l'entierté des mots composants le terme -> ['matière','premier]
                        #-> dans ce cas c'est correct
                    #B : l'élève n'a trouvé qu'un des mots composant le mot  -> ['matière']
                        #-> dans ce cas on indique qu'il manque une partie du mot OU on ne le prends juste pas en compte et on l'indique comme incorrect (ce qu'on fait ici)
                    for el in element : 
                        if el not in eq_e : 
                            element_trouvee = False
                    if element_trouvee != False : 
                        for el in element :
                            if el not in termes_corrects:
                                termes_corrects.append(el)
                        element_trouve = True
                        break

            
            # Si aucun élément n'a été trouvé dans la sous-liste, ajouter la sous-liste à la liste des éléments manquants
            if not element_trouve:
                if sous_liste[0] not in manque :
                    manque.append(sous_liste[0])
        #si les termes corrects ont correctement été retrouvés:                    
        if len(termes_corrects)>0:
            RESULTAT.append('Ces termes important ont correctement été retrouvés :')
            for i in termes_corrects: 
                RESULTAT.append(i)
                if p_equivalence == True:
                    retour2.append(f"'{terme}' corresponds bien à un ensemble de matières premières.")
                    retour2.append(f"ou : '{terme}' corresponds bien à '{termes_corrects}'")
                elif p_possession == True:
                    retour2.append(f" '{terme}' est bien possédé par une entreprise.")
                    retour2.append(f"ou : '{terme}' est bien possédé par '{termes_corrects}'")
                elif p_but == True :
                    retour2.append(f"Le but du '{terme}' est bien la vente.")
                    retour2.append(f"ou : Le but du '{terme}' est bien '{termes_corrects}'")
                
                
        if len(manque)>0:
            retour2.append('Ces termes important manquent :')
            RESULTAT.append('Ces termes important manquent :')
            for i in manque: 
                retour2.append(i)
                RESULTAT.append(i)

        #si les termes n'ont pas été correctement retrouvés : 
        elements_extras = [element for element in eq_e if element not in [element for element in termes_corrects]]
        
        #ici pour les antonymes
        if len (elements_extras) > 0:
            RESULTAT.append('Ces termes ont été incorrectement retrouvés :')
            #RESULTAT.append(elements_extras[0])-> retiré 08/09
            for i in elements_extras :
                if i not in RESULTAT:
                    RESULTAT.append(i)


        #suppression anto & exemples ici 




    #deuxième possibilité
    if len(eq_p_important)>0 and len(eq_e) == 0 : 

        RESULTAT.append(f"Il manque la relation '{relation_txt}'")
        #On observe alors les mots-clefs
        liste_mot_eleve = nom_et_verbe(definition_e)
        for i in liste_mot_eleve : 
            for j in eq_p_important:
                for k in j : 
                    if type(k) != list : 
                        if i == k:
                            if k not in termes_corrects :
                                termes_corrects.append(k)
                    else :#pour ex [matière,premier]
                        liste_k_copie = k[:]
                        for longueur in k : 
                            if longueur in liste_mot_eleve:
                                liste_k_copie.remove(longueur)
                        if len(liste_k_copie)==0:
                            if k not in termes_corrects :
                                termes_corrects.append(k)
                                break
                            
        if len(termes_corrects)>0:
            RESULTAT.append(f"Toutefois, certains mots faisant bien partie de la relation '{relation_txt}' sont présents, il faudrait donc vérifier la définition. Les termes sont :")
            for i in termes_corrects:
                RESULTAT.append(i)
                if p_equivalence == True:
                    retour2.append(f"'{terme}' corresponds bien à un ensemble de matières premières.")
                    retour2.append(f"ou : '{terme}' corresponds bien à '{termes_corrects}'")
                elif p_possession == True:
                    retour2.append(f" '{terme}' est bien possédé par une entreprise.")
                    retour2.append(f"ou : '{terme}' est bien possédé par '{termes_corrects}'")
                elif p_but == True :
                    retour2.append(f"Le but du '{terme}' est bien la vente.")
                    retour2.append(f"ou : Le but du '{terme}' est bien '{termes_corrects}'")
        
        else : 
            if p_equivalence == True:
                retour2.append(f"Il est important de noter que '{terme}' corresponds à '{eq_p_important}',un ensemble de matières premières.")
                
            elif p_possession == True:
                retour2.append(f"Il est important de noter que '{terme}' est possédé par une entreprise.")
            elif p_but == True :
                retour2.append(f"Il est important de noter que le but du '{terme}' est la vente.")
           

    #troisième possibilité
    if len(eq_p_important)==0 and len(eq_e) > 0 : 
        RESULTAT.append(f"La relation '{relation_txt}' est superflue")

    return RESULTAT
    #return retour2


def correction2 (definition_correcte,definition_eleve,definition_e,terme=None):
    """
    @input:
        la définition correcte du terme défini sous forme de dictionnaire
        la définition de l'élève sous forme de dictionnaire
        la définition sous forme de string
    @output:appels à la fonction "partie1correction" pour chacune des relations
    """
    #partie equivalence
    A = correction_SC(definition_correcte,definition_eleve,definition_e)

    #partie possession 
    B = correction_SC(definition_correcte,definition_eleve,definition_e,"POSSESSION","de possession")

    #partie but 
    D = correction_SC(definition_correcte,definition_eleve,definition_e,"BUT","de but")
    
    return(A,B,D)
        
#definition_eleve = { 'BUT': {'S': ['stock'], 'O': ['vendre', 'acheter']}}
#definition_e = "bien que l’on possède, qui comporte une certaine masse, c’est un élément saisissable, il n’est donc pas virtuel ( ex : des jouets ( marchandise ) d’une entrepise sont un stock, tant dit qu’un service ne l’ait pas.)."

#def_corr = retour[0]
#print(retour)
#correction2(def_corr,definition_eleve,definition_e)

def correction_eleve (dictionnaire_synonymes,dico,fichier_definition_e,fichier_correction,terme="stock"):
    """
    @input : dictionnaire contenant les synonymes 
             dictionnaire contenant les définitions du professeurs
             un fichier contenant les définitions des élèves, à corriger
             un nom de fichier qui contiendra les définitions 
    @output : un fichier avec les définitions accompagnées de leurs corrections
    @durée : 15.6s
    """
    j=1
    liste_de_lignesinfichier = []
    liste_de_lignes = []
    liste_a_ecrire = []
    with open (fichier_definition_e, "r", encoding = 'utf-8') as fichier : 
        for lignes in fichier: 
            liste_de_lignesinfichier.append(lignes)



    for lignes in liste_de_lignesinfichier :
        lines = [lignes]
        lines[-1] = lines[-1].rstrip('\n')
        for line in lines : 
            liste_de_lignes.append(line)


    #SCleplusproche 
    listeee = list(dico.values())
    listee = listeee[0]

    for line in liste_de_lignes : 
        definition_e = line
        liste_a_ecrire.append(f"\n'{j}'"+line+"\n")
        j+=1

        definition_eleve = representation_liste_dictionnaire_e(terme,definition_e) #scàcorriger
        SCleplusproche = trouver_SC("stock",listee,definition_e)
        SCleplus_proche = SCleplusproche[0]
        #SCleplus_proche = construire_representation_dictionnaire_(dictionnaire_synonymes, SCleplusproche)

        corr = correction2(SCleplus_proche,definition_eleve,definition_e,terme)
        if len(corr) != 0 :
            for i in corr :
                liste_a_ecrire.append(f"'{i}'\n")



    with open(fichier_correction, "w", encoding= 'utf-8') as pli :
        for ecrire in liste_a_ecrire : 
            pli.write(ecrire)
            
#durée:2m30


In [ ]:
#SUPP
dico_eleve = [
"C’est un nombre de  biens que l’on possède, qui ne sont pas encore vendus ou consommés."
,"Bien ou matière première dont dispose l’entreprise qui va servir pour la production ou vendu dans l’état.  "
,"Un stock représente les biens d’une entreprise qui peuvent être destinés à la vente dans le cadre de l’activité de l’entreprise. ex: Un stock de tables ou de bois que l’entreprise vend directement  ( stocks de marchandise et matière premières.) Ou pour être consommés pour créer d'autres biens. ex: Un stock de bois utilisé pour créer une table ( stocks de produits finis.)"
,"ce sont des biens achetés et entreposés au cas où il y aurait un manque, c’est un effet de «réserve»"
,"Ensemble de biens / matières premières pouvant etre stockés avant d’être vendu en produit fini ou non."
,"Le stock correspond à l'ensemble des produits qu'une entreprise possède, stocké en attendant leurs utilisations ou leurs ventes."
,"ensemble de marchandise disponible"
,"Quantité de marchandise en réserve"
,"Produits restants après une vente."
,"Le stock  correspond à la marchandise entreposé dans le magasin"
,"Un stock correspond à des biens restants après une vente"
]

fichier_definition_e = "stock_def_eleves.txt"
fichier_correction = 'stock_def_elevescorrige_SCCOR2.txt'
correction_eleve(dictionnaire_synonymes,dico,fichier_definition_e,fichier_correction)

In [32]:

def sujet_possession(terme,definition_correcte,definition_eleve):
    """
    cette fonction nous permets de vérifier si le sujet de la relation de possession est bien le terme définit, nous imprimons le résultat
    @input : le terme définit, la définition correcte sous forme de dictionnaire et la définition de l'élève, également sous forme de dictionnaire
    @output : /
    """

    pre = 0
    for cle in definition_eleve.keys():
        if cle == "EQUIVALENCE":
            pre += 1
            #print(definition_eleve["EQUIVALENCE"])
        if cle == "POSSESSION":  
            pre += 1
            #print(definition_eleve["POSSESSION"])

    for cle in definition_correcte.keys():
        if cle == "EQUIVALENCE":
            pre += 1
            #print(definition_correcte["EQUIVALENCE"])
        if cle == "POSSESSION":  
            pre += 1
            #print(definition_correcte["POSSESSION"])
    #print(pre)        
    if pre == 4 :
        if (definition_eleve["POSSESSION"]["S"]) == (definition_eleve["EQUIVALENCE"]["O"]):
            return("Le sujet de la possession est bien le terme défini")

    
        sujet_poss = (definition_eleve["POSSESSION"]["S"])
        sujet_eq = (definition_correcte["EQUIVALENCE"]["S"])
        eq_p_important = (definition_correcte["EQUIVALENCE"]["O"])
        #eq_p_important = (definition_correcte["EQUIVALENCE"]["O"]["important"])
        #soit le sujet de l'équivalence
        if sujet_poss == sujet_eq : 
            return (f"Le sujet de la possession '{element}' est correct")
            
        #soit dans les éléments importants de l'équivalence
        for el in eq_p_important :
            for element in sujet_poss :
                if element in el :
                    return(f"Le sujet de la possession '{element}'est correct")
                        
        sujet_poss = (definition_eleve["POSSESSION"]["S"])
        if sujet_poss == terme :
            return (f"Le sujet de la possession '{terme}' est correct")    
        else : 
            return (f"Le sujet de la possession '{sujet_poss}' ne semble pas être correct")


In [ ]:
#Importer listes d'antonymes
from liste_synonymes import dico_anto
dictionnaire_antonymes = dico_anto()

liste_fichier_antonyme = list(dictionnaire_antonymes.keys())
def antonyme_etape_1 (liste_sujet_ou_objet, liste_fichier_antonyme=liste_fichier_antonyme):
    liste_antonymes = [] #pour faire le correctif_antonyme
    #Maintenant, on reprends les sujets : 
    for i in range(len(liste_sujet_ou_objet)) : 
        liste1 = liste_sujet_ou_objet[i]
        for j in liste1 : 
            if j in liste_fichier_antonyme : 
                liste_antonymes.append(j)

    return(liste_antonymes)

def antonyme_dictionnaire (dictionnaire_correctif,dictionnaire_antonymes):
    """
    @input : le dictionnaire_correctif, le dictionnaire de liste d'antonymes
    exemple : {'EQUIVALENCE': {'S': ['stock'], 'O': [['ensemble', 'réserve', 'quantité', 'nombre', 'accumulation'], ['produit', 'article', 'élément', 'bien', 'actif', 'avoir', 'ressource'], ['physique'], ['agrégat'], ['article']]},
    'POSSESSION': {'S': [['stock'],['ensemble', 'réserve', 'quantité', 'nombre', 'accumulation'], ['produit', 'article', 'élément', 'bien', 'actif', 'avoir', 'ressource'], ['physique'], ['agrégat'], ['article']],'O': [['entreprise', 'organisation', ['organisme', 'activité', 'marchand']]]}, 
    'BUT': {'S': 'stock', 'O': [['utilisation', 'utiliser'], ['vente', 'vendre', 'revendre', 'revente', 'commercialisation', 'cession'], ['utiliser'], ['processus', 'fonctionnement', ['pilotage', 'quotidien']], ['produire', 'production'], ['vendre'], ['marché']]}}

    @output : un dictionnaire d'antonyme pour le correctif
    """
    dico = dictionnaire_correctif
    dico_anto = {}
    for i in range (len(dico.keys())):
        s_o = {}#pour le dictionnaire
        liste_sujet_antonyme = []
        liste_objet_antonyme=[]

        key = (list(dico.keys())[i])

        #sujet
        sujet = (dico[key]['S'])
        sujet_antonyme = antonyme_etape_1(sujet)


        if len(sujet_antonyme) == 1 : #pour ne pas avoir [[]]
            liste_sujet_antonyme = dictionnaire_antonymes[sujet_antonyme[0]]
        else : 
            for i in sujet_antonyme : 
                liste_sujet_antonyme.append(dictionnaire_antonymes[i])

        s_o["S"]=liste_sujet_antonyme#pour le dictionnaire

        #objet
        objet = (dico[key]['O'])
        objet_antonyme = antonyme_etape_1(objet)

        if len(objet_antonyme) == 1 : 
            liste_objet_antonyme = dictionnaire_antonymes[objet_antonyme[0]]
        else : 
            for i in objet_antonyme : 
                liste_objet_antonyme.append(dictionnaire_antonymes[i])

        s_o["O"]=liste_objet_antonyme
        
        dico_anto[key]=s_o


    return(dico_anto)

#print(anto_dico)
#exemple retour : 
#{'POSSESSION': {'S': ['service', 'passif', 'dette', ['obligation', 'financier'], 'endettement'], 'O': []}, 'BUT': {'S': [], 'O': ['stocker', 'stockage', 'garder', 'emmagasiner', 'accumuler', 'conserver', 'acheter', 'achat']}}
